# Extract YRBS Data

Youth Risk Behavior Survey (YRBS) data has been converted from fixed-width-format, to comma separated values.  The dataset was split into two by state name, A-M and N-Z. In this notebook, we will merge the two together and perform some basic cleanup and initial exploration.

This notebook saves results in local SQL database table `STATE`.


In [1]:
import sys
import time
import pandas as pd
import numpy as np
from pathlib import Path
from sqlalchemy import inspect, create_engine, Table, Column, Integer, String

start_time = time.time()

## Set the file paths and read data

TODO:  Review other csv files and find another cloud storage option.

Download the converted CSV files from here: https://drive.google.com/file/d/11KfEMD5GrSDqIwB6jURa45W0fX2yezXF/view?usp=sharing

Then extract the contents of the zip file into the `data` directory. You should have four csv files:

        <project root>
        .
        ├── LICENSE
        ├── README.md
        ├── data
        │   └── README.md
        ├── notebooks
        │   ├── 01_Extract.ipynb
        │   ├── 02_Transform.ipynb
        │   ├── 02_Transform_Chicago.ipynb
        │   └── 02_Transform_NYC.ipynb
        ├── origin_data
        │   ├── README.md
        │   ├── SADC_District_Q.csv
        │   ├── SADC_National_Q.csv
        │   ├── SADC_State_a_m_Q.csv
        │   └── SADC_State_n_z_Q.csv
        └── src
            ├── connection.r
            ├── dictionaries.py
            ├── labels.py
            ├── test.py
            └── utils.py
---


In [2]:
state_a_m_datafile = Path("../origin_data/SADC_State_a_m_Q.csv")
state_n_z_datafile = Path("../origin_data/SADC_State_n_z_Q.csv")
district_datafile = Path("../origin_data/SADC_District_Q.csv")
national_datafile = Path("../origin_data/SADC_National_Q.csv")
state_a_m_df = pd.read_csv(state_a_m_datafile)
state_n_z_df = pd.read_csv(state_n_z_datafile)
district_df = pd.read_csv(district_datafile)
national_df = pd.read_csv(national_datafile)
state_df = pd.concat([state_a_m_df, state_n_z_df])


## Data Cleanup

In [3]:
# convert `year` to int
district_df["year"] = district_df["year"].astype('int')
district_df["year"] = district_df["year"].astype('int')
national_df["year"] = national_df["year"].astype('int')


## Address any NaN values



In [4]:
# Convert categorized `object` data to int and fillna() with `0`
"""NOTE:
Convert these categorical columns to int and fill null values with 0.

"""
cols_int = ["sitetypenum", "survyear", "stratum", "PSU", "age", "sex", "grade", "race4", "race7", "qnobese", "qnowt", "q66", "q65", "sexid", "sexid2", "sexpart", "sexpart2", 
            "q8", "q9", "q10", "q11", "q12", "q13", "q14", "q15", "q16", "q17", "q18", "q19", "q20", "q21", "q22", "q23", "q24", "q25", "q26", "q27", "q28", "q29", 
            "q30", "q31", "q32", "q33", "q34", "q35", "q36", "q37", "q38", "q39", "q40", "q41", "q42", "q43", "q44", "q45", "q46", "q47", "q48", "q49", 
            "q50", "q51", "q52", "q53", "q54", "q55", "q56", "q57", "q58", "q59", "q60", "q61", "q62", "q63", "q64", "q67", "q68", "q69", 
            "q70", "q71", "q72", "q73", "q74", "q75", "q76", "q77", "q78", "q79", "q80", "q81", "q82", "q83", "q84", "q85", "q86", "q87", "q88", "q89", 
            "qbikehelmet", "qdrivemarijuana", "qcelldriving", "qpropertydamage", "qbullyweight", "qbullygender", "qbullygay", "qchokeself", "qcigschool", 
            "qchewtobschool", "qalcoholschool", "qtypealcohol2", "qhowmarijuana", "qmarijuanaschool", "qcurrentopioid", "qcurrentcocaine", "qcurrentheroin", "qcurrentmeth", 
            "qhallucdrug", "qprescription30d", "qgenderexp", "qtaughtHIV", "qtaughtsexed", "qtaughtstd", "qtaughtcondom", "qtaughtbc", "qdietpop", "qcoffeetea", "qsportsdrink", 
            "qenergydrink", "qsugardrink", "qwater", "qfastfood", "qfoodallergy", "qwenthungry", "qmusclestrength", "qsunscreenuse", "qindoortanning", "qsunburn", "qconcentrating", 
            "qcurrentasthma", "qwheresleep", "qspeakenglish", "qtransgender"]

for col in cols_int:
    state_df[col] = state_df[col].fillna(0)
    state_df[col] = state_df[col].astype('int')
    district_df[col] = district_df[col].fillna(0)
    district_df[col] = district_df[col].astype('int')
    national_df[col] = national_df[col].fillna(0)
    national_df[col] = national_df[col].astype('int')


## Save Imported Data to SQL Database Tables

In [5]:
connection_string = 'sqlite:///../data/yrbss_data.db'
engine = create_engine(connection_string)
insp = inspect(engine)


['DISTRICT', 'NATIONAL', 'STATE']
duration: 733.5140371322632


In [ ]:
start_time = time.time()
state_df.to_sql('STATE', engine, index=False, if_exists='replace')
# print(insp.get_table_names())
# display(pd.read_sql_query("SELECT * FROM STATE LIMIT 3;", con=engine))
# display(pd.read_sql_query("SELECT COUNT() FROM STATE", con=engine))
current_time = time.time()
elapsed_time = current_time - start_time
print(f"STATE import took {int(elapsed_time)} seconds")

In [ ]:
start_time = time.time()
district_df.to_sql('DISTRICT', engine, index=False, if_exists='replace')
# print(insp.get_table_names())
# display(pd.read_sql_query("SELECT * FROM STATE LIMIT 3;", con=engine))
# display(pd.read_sql_query("SELECT COUNT() FROM STATE", con=engine))
current_time = time.time()
elapsed_time = current_time - start_time
print(f"DISTRICT table import took {int(elapsed_time)} seconds")

In [9]:
start_time = time.time()
national_df.to_sql('NATIONAL', engine, index=False, if_exists='replace')
# display(pd.read_sql_query("SELECT * FROM STATE LIMIT 3;", con=engine))
# display(pd.read_sql_query("SELECT COUNT() FROM STATE", con=engine))
current_time = time.time()
elapsed_time = current_time - start_time
print(f"NATIONAL table import took {int(elapsed_time)} seconds")
print(insp.get_table_names())

NATIONAL table import took 41 seconds
['DISTRICT', 'NATIONAL', 'STATE']


## Read Sample Data into Pandas

In [11]:
for t in metadata_obj.sorted_tables:
    print(t.name)

NameError: name 'metadata_obj' is not defined

In [6]:
# my_subset_df = pd.read_sql_query("SELECT `year`, `age` FROM STATE ;", con=engine)
# my_subset_df.shape

In [7]:
# print(f"Imported from SQL Dataframe Shape: {my_subset_df.shape}\n\n")
# print("Head:")
# display(my_subset_df.head(3))
# print("Tail:")
# display(my_subset_df.tail(3))